In [1]:
import re
import pandas as pd
import os
import itertools
import json
import evaluate

/home/michele.maggini/miniconda3/envs/unsloth_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
cd = '/home/michele.maggini/Llama3/'
filename = 'CLEF3A_sys_ans_CoT.csv'
path = os.path.join(cd, filename)


filename_2 = "CLEF3A_sampled.csv"
path_2 = os.path.join(cd, filename_2)
df_gt = pd.read_csv(path_2)

#n_example=20
#df = df.groupby('label').head(n_example)

df_ans = pd.read_csv(path)

In [3]:
df_gt.shape, df_ans.shape

((3000, 2), (3000, 1))

In [4]:
def extract_last_line(text):
    # Split the text into lines
    lines = text.strip().split('\n')
    # Return the last line
    return lines[-1]


def extract_integers(text):
    # Regular expression to match integers enclosed in single quotes
    pattern = r"'(\d+)'"
    
    # Find all matches in the text
    matches = re.findall(pattern, text)
    
    # Convert matches to integers
    integers = [int(match) for match in matches]
    
    return integers


def extract_numbers(string_list):
    """
    This function takes a list of strings, each representing a number,
    and returns a new list of integers.

    :param string_list: List of strings representing numbers
    :return: List of integers
    """
    return [int(num) for num in string_list]

In [5]:
listLastline = df_ans['0'].apply(lambda x: extract_last_line(x))
df_lastline = listLastline.to_list()
df_lastline[0]

"Based on the analysis, I would label this headline as a right-wing hyperpartisan article. The language used is critical of Kamala Harris, a prominent Democratic politician, and the tone is sensational and emotive. The article's intent appears to be to persuade rather than inform, and the language used is likely to appeal to a right-wing audience."

In [6]:
def extract_integers(input_list):
    # This will store the extracted integers
    integers = []
    
    # Define a regex pattern to match integers
    integer_pattern = re.compile(r'\b\d+\b')
    
    for index, item in enumerate(input_list):
        # Find all integers in the string item
        found_integers = integer_pattern.findall(item)
        
        if found_integers:
            # Convert found integers to int and add to the result list
            integers.extend(map(int, found_integers))
        else:
            # Check for keywords if no integers are found
            item_lower = item.lower()
            if 'left' in item_lower:
                integers.append(0)
            elif 'center' in item_lower or 'neutral' in item_lower:
                integers.append(1)
            elif 'right' in item_lower:
                integers.append(2)
            else:
                # Print the index if neither integers nor keywords are found
                print(f"No integer or keyword found in element at index {index}", integers.append(1))
    
    return integers

In [7]:
df_lastline[136]

'This indicates that the article is not hyperpartisan, as it does not exhibit any strong ideological bias or attempt to persuade the reader.'

In [8]:

gold_ans = df_gt['label'].tolist()
print("gold_ans:", gold_ans)
labels_parsed = extract_integers(df_lastline)

gold_ans: [2, 2, 1, 2, 1, 2, 2, 0, 2, 2, 0, 2, 0, 1, 1, 0, 2, 1, 0, 0, 1, 1, 0, 2, 1, 1, 1, 0, 0, 2, 0, 1, 2, 1, 0, 1, 1, 1, 1, 2, 0, 1, 2, 0, 1, 2, 2, 1, 1, 2, 0, 2, 2, 0, 1, 2, 1, 1, 1, 0, 2, 2, 2, 0, 1, 2, 1, 1, 2, 2, 0, 0, 0, 2, 2, 2, 0, 2, 2, 0, 0, 0, 2, 1, 2, 1, 2, 2, 1, 2, 2, 2, 1, 0, 0, 1, 2, 1, 0, 1, 1, 1, 2, 2, 1, 2, 2, 2, 0, 0, 0, 2, 2, 2, 1, 1, 2, 0, 2, 0, 0, 2, 0, 0, 1, 1, 0, 2, 2, 2, 2, 2, 2, 1, 0, 1, 0, 2, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 2, 1, 0, 0, 2, 1, 0, 2, 2, 1, 1, 2, 1, 1, 1, 2, 2, 0, 1, 0, 2, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 0, 0, 0, 2, 2, 2, 1, 1, 0, 1, 2, 1, 0, 2, 0, 1, 0, 2, 2, 1, 1, 2, 0, 1, 0, 2, 2, 2, 0, 2, 2, 1, 2, 0, 1, 2, 1, 1, 2, 2, 2, 1, 0, 2, 1, 0, 1, 1, 0, 2, 0, 2, 2, 1, 1, 2, 0, 2, 1, 2, 2, 0, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 0, 2, 0, 0, 1, 0, 0, 2, 1, 1, 2, 1, 0, 1, 1, 2, 2, 0, 2, 0, 2, 1, 2, 2, 1, 1, 1, 2, 0, 2, 1, 2, 0, 0, 0, 1, 1, 2, 2, 1, 0, 1, 1, 0, 2, 2, 1, 1, 2, 2, 0, 1, 2, 2, 0, 1, 1, 1, 2, 2, 2, 1, 1, 0, 2, 2, 1, 1, 0, 2, 2,

In [9]:
len(labels_parsed)

3000

In [10]:
def compute_metrics(predictions, labels):
        accuracy_metric = evaluate.load("accuracy")
        precision_metric = evaluate.load("precision")
        recall_metric = evaluate.load("recall")
        f1_metric = evaluate.load("f1")


        accuracy = accuracy_metric.compute(predictions=predictions, references=labels)[
            "accuracy"
        ]
        precision = precision_metric.compute(
            predictions=predictions, references=labels, average="weighted"
        )["precision"]
        recall = recall_metric.compute(
            predictions=predictions, references=labels, average="weighted"
        )["recall"]
        f1 = f1_metric.compute(
            predictions=predictions, references=labels, average="weighted"
        )["f1"]

        return {
            "accuracy": accuracy,
            "precision": precision,
            "recall": recall,
            "f1-score": f1,
        }


results=compute_metrics(labels_parsed, gold_ans)
print(f"CLEF3A, CoT, prompt with temperature=0.1:\n", json.dumps(results, indent=4))

CLEF3A, CoT, prompt with temperature=0.1:
 {
    "accuracy": 0.4553333333333333,
    "precision": 0.4848061501002057,
    "recall": 0.4553333333333333,
    "f1-score": 0.44758415114104694
}


/home/michele.maggini/miniconda3/envs/unsloth_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [11]:
from sklearn.metrics import precision_recall_fscore_support, classification_report

print(classification_report(gold_ans, labels_parsed))

              precision    recall  f1-score   support

           0       0.45      0.44      0.45      1000
           1       0.41      0.62      0.50      1000
           2       0.59      0.30      0.40      1000
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         0
           7       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         0

    accuracy                           0.46      3000
   macro avg       0.21      0.20      0.19      3000
weighted avg       0.48      0.46      0.45      3000



/home/michele.maggini/miniconda3/envs/unsloth_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/michele.maggini/miniconda3/envs/unsloth_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/michele.maggini/miniconda3/envs/unsloth_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier